In [1]:
import pandas as pd

In [2]:
source_comments = pd.read_csv('./wall/girl_wall/comments/test_20.02.19-13:49_girl.csv', index_col=0)
source_answers = pd.read_csv('./wall/girl_wall/answers/test_20.02.19-13:49_girl.csv', index_col=0)
source_posts = pd.read_csv('./wall/girl_wall/posts/test_20.02.19-13:49_girl.csv', index_col=0)
source_comments.replace({pd.np.NaN: None}, inplace=True)
source_answers.replace({pd.np.NaN: None}, inplace=True)
source_posts.replace({pd.np.NaN: None}, inplace=True)
source_comments.head()

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
0,4212,1549722387,520143737,0,5385,[],4211,None,None,Зачем туда идти 😊
1,4206,1547927323,4014996,1,5385,[],4205,None,None,"Опасные русские медведи, о которых ходят леген..."
2,4214,1550430898,200531929,0,5385,[],4202,None,None,Мне такое чудо нужен!
3,4215,1550473100,5385,0,5385,[],4202,None,None,Котик уже нашел дом👌👌👌
4,4203,1547846810,525460657,0,5385,[],4192,None,None,Круто)


In [3]:
source_comments.sort_values(by=['post_id', 'date'] , inplace=True)
source_answers.sort_values(by=['post_id', 'date'] , inplace=True)
source_answers.head()

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
2,4169,1550421347,6154,0,6154,[4167],3466,None,None,"[id51816062|Марина], сейчас да, раньше просто ..."
34,3876,1544340697,10210,0,10210,[3875],3731,None,None,Стараюсь)
31,3910,1547728110,10210,0,10210,[3909],3848,None,None,"[id3171305|Роман], ато:) приедешь посмотришь ))"
32,3911,1547729010,3171305,0,10210,[3909],3848,None,None,Потискаю))))
33,3912,1547729245,10210,0,10210,[3909],3848,3911,3.17130e+06,"[id3171305|Роман], договорились:)"


In [66]:
#base
from json import JSONDecoder
j = JSONDecoder()

already_used_id = []
dialogs = []


answers = source_answers.copy()
anchor = answers[list(map(lambda x: x!='[]', answers['parents_stack']))].head(1)
while not anchor.empty:
    dialog = []
    already_used_id.append(int(anchor['comment_id']))
    
    parent_id = j.decode(answers['parents_stack'].iloc[0])[0]
    dialog.append(source_comments[source_comments['comment_id'] == parent_id].head(1))
    dialog.append(anchor)
    
    after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    while not after_anchor.empty:
        anchor = after_anchor
        already_used_id.append(anchor['comment_id'])
        dialog.append(anchor)
        after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    
    if len(pd.concat(dialog)) > 1:
        dialogs.append(pd.concat(dialog))
    
    answers = answers[~answers['comment_id'].isin(already_used_id)]
    anchor = answers[list(map(bool, answers['parents_stack']))].head(1)

In [67]:
dialogs[3]

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
736,3909,1547728051,3171305,3,10210,[],3848,None,None,Коасавчек
32,3911,1547729010,3171305,0,10210,[3909],3848,None,None,Потискаю))))
33,3912,1547729245,10210,0,10210,[3909],3848,3911,3.17130e+06,"[id3171305|Роман], договорились:)"


In [70]:
def post_processing(dialogs):
    res=[]
    dialog=[]
    dialog.append(dialogs[0])
    for cur in dialogs[1:]:
        if set(dialog[-1]['from_id']) == set(cur['from_id'])\
            and set(dialog[-1]['post_id']) == set(cur['post_id']):
            print('est')
            dialog.append(cur)
        else:
            if len(dialog) > 1:
                head = dialog[0]
                tail = pd.concat(dialog[1:])
                tail = tail[tail['parents_stack']!='[]']
                res.append(pd.concat([head, tail]))
            else:
                res.append(pd.concat(dialog))
            dialog=[cur]
    return res
new_dialogs = post_processing(dialogs)

est
est
est


In [80]:
new_dialogs[2].to_dict()

{'comment_id': {736: 3909, 31: 3910, 32: 3911, 33: 3912},
 'date': {736: 1547728051, 31: 1547728110, 32: 1547729010, 33: 1547729245},
 'from_id': {736: 3171305, 31: 10210, 32: 3171305, 33: 10210},
 'num_answer': {736: 3, 31: 0, 32: 0, 33: 0},
 'owner_id': {736: 10210, 31: 10210, 32: 10210, 33: 10210},
 'parents_stack': {736: '[]', 31: '[3909]', 32: '[3909]', 33: '[3909]'},
 'post_id': {736: 3848, 31: 3848, 32: 3848, 33: 3848},
 'reply_to_comment': {736: None, 31: None, 32: None, 33: 3911.0},
 'reply_to_user': {736: None, 31: None, 32: None, 33: 3171305.0},
 'text': {736: 'Коасавчек',
  31: '[id3171305|Роман], ато:) приедешь посмотришь ))',
  32: 'Потискаю))))',
  33: '[id3171305|Роман], договорились:)'}}

In [108]:
def dialogs_to_json(dialogs, path_to_save):
    from json import JSONEncoder
    
    j = JSONEncoder()
    all_dialogs = []
    persona_id = 0
    other_id = 0
    
    for num, i in enumerate(dialogs):
        dialog = []
        for _, row in i.iterrows():
            if row['from_id']==row['owner_id']:
                persona_type = 'persona'
                persona_id = int(row['from_id'])
            else:
                persona_type = 'other'
                other_id = int(row['from_id'])
            utterance = {
                "persona_type": persona_type,
                "num_answers": int(row['num_answer']),
                "text": row['text']
            }
            dialog.append(utterance)
        
        post = source_posts[source_posts['post_id'] == int(i['post_id'].iloc[0])].head(1)
        curr_dialog = {
            "dialog_id": f"test_{str(num)}",
            "post_text": post['text'].iloc[0],
            "post_num_comments": int(post['num_comments'].iloc[0]),
            "persona_id": persona_id,
            "other_id": other_id,
            "dialog": dialog
        }
        all_dialogs.append(curr_dialog)
    with open(path_to_save, 'w') as fw:
        fw.write(j.encode(all_dialogs))
        

In [109]:
dialogs_to_json(new_dialogs, './test/testing_json_file.json')

In [74]:
def view_dialog(table):
    if len(table) > 1:
        res = []
        for _, i in table.iterrows():
            if i['text']:
                res.append(i['text'])
            else:
                return
        return res

In [76]:
with open('./view_answers_post_proc.txt', 'w') as fw:
    for num, i in enumerate(new_dialogs):
        a = view_dialog(i)
        if a:
            for line in a:
                fw.write(line+'\n')
        fw.write(str(num)+"   "+'*'*10+'\n\n\n\n'+"*"*10+'\n')

In [22]:
import time
time.ctime(1520761320)

'Sun Mar 11 12:42:00 2018'